In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer

# Data base import

In [ ]:
from utils import sample_data

NB_SAMPLES = 10
PATH_SAMPLE = Path("../data/samples/sample_{}.csv".format(NB_SAMPLES))
PATH_CSV = Path("../data/train.csv")

SMALL_DATA = sample_data.SampleData(
    PATH_CSV, nb_samples=NB_SAMPLES,
    save=True, path_save=PATH_SAMPLE
)

DATAFRAME = pd.read_csv(PATH_SAMPLE)
DATA = pd.DataFrame.to_numpy(DATAFRAME)

In [ ]:
print(DATAFRAME)

# Pre-processing

In [ ]:
from utils.load_data import load_data
from utils.data_descriptor import convert_labels

ALPHANUM_ONLY = False
WORD_SIZE = 30
SENTENCE_SIZE = 50
FILL_WITH = "$"
SPLIT_PUNCTUATION = False # to tell wheter the puncutation "!?.;,/" etc are keeped sticked to a word or not

X_STRING, X_SCALAR = load_data(DATA, WORD_SIZE, SENTENCE_SIZE, FILL_WITH, SPLIT_PUNCTUATION, ALPHANUM_ONLY)

Y = convert_labels(DATA, SENTENCE_SIZE, SPLIT_PUNCTUATION)

print(DATA.shape, X_STRING.shape, Y.shape, X_SCALAR.shape)

print("\nOriginal data :")
print(DATA[0])
print("\nFilled sentence :")
print(X_STRING[0])
print("\nLabel :")
print(Y[0])
print("\nDescriptor :")
print(X_SCALAR[0])

In [ ]:
X_ORIGINAL = DATA[:, 1]

In [ ]:
X_SCALAR.shape

# Classification

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
nb_neighbors = 1

knn = KNeighborsRegressor(nb_neighbors, weights="distance")
knn.fit(X_SCALAR, Y)

In [ ]:
predictions = knn.predict(X_SCALAR)

In [ ]:
from utils.post_processing import pred_to_string

results = np.zeros(len(predictions), dtype=object)
for i in range(len(predictions)):
    results[i] = pred_to_string(X_ORIGINAL[i], X_STRING[i], predictions[i])

In [ ]:
for i in range(len(results)):
    print(DATA[i][1])
    print(DATA[i][2])
    print(results[i])
    print()

In [ ]:
from utils.loss import jaccard

avg = 0
for i in range(len(results)):
    avg += jaccard(results[i], DATA[i, 2])
    
    if jaccard(results[i], DATA[i, 2]) != 1:
        print("Error on", i)
avg /= len(results)

print(avg)

In [ ]:
print(DATA[3][2])

for i in range(len(X_STRING[3])):
    print()
    print(X_STRING[3][i])
    print(Y[3][i])